In [103]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [104]:
np.load('../0.npy', allow_pickle=True)

array([ 0.51613885,  0.5281536 , -0.65717638, ...,  0.        ,
        0.        ,  0.        ])

In [112]:
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('../collected_data')

actions = []
actions.sort()
#Actions that we try to detect
for root, dirs, files in os.walk(DATA_PATH):
    for dir_name in dirs:
        if not dir_name.isdigit():
            actions.append(dir_name)

actions = np.array(actions)
print(actions)

#30 videos worth of data
no_sequence = 30

#Videos are going to be 30 frames in length
sequence_length = 30

['hello' 'is' 'my' 'name']


In [113]:
label_map = {label:num for num, label in enumerate(actions)}

In [114]:
label_map

{'hello': 0, 'is': 1, 'my': 2, 'name': 3}

In [115]:
sequences, label = [], []
for action in actions:
    for sequence in range(no_sequence):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)), allow_pickle=True)
            window.append(res)
        sequences.append(window)
        label.append(label_map[action])

In [116]:
np.array(sequences).shape

(120, 30, 1662)

In [117]:
np.array(label).shape

(120,)

In [118]:
X = np.array(sequences)
y = to_categorical(label).astype(int)

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [120]:
# Python: Launch TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [122]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(actions.shape[0], activation='softmax'))

In [123]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [124]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
4/4 [==============================] - 3s 77ms/step - loss: 2.0329 - categorical_accuracy: 0.2456
Epoch 2/100
4/4 [==============================] - 0s 75ms/step - loss: 1.4949 - categorical_accuracy: 0.2807
Epoch 3/100
4/4 [==============================] - 0s 75ms/step - loss: 2.3610 - categorical_accuracy: 0.3070
Epoch 4/100
4/4 [==============================] - 0s 83ms/step - loss: 1.5254 - categorical_accuracy: 0.2281
Epoch 5/100
4/4 [==============================] - 0s 79ms/step - loss: 1.4389 - categorical_accuracy: 0.2895
Epoch 6/100
4/4 [==============================] - 0s 73ms/step - loss: 1.2051 - categorical_accuracy: 0.4737
Epoch 7/100
4/4 [==============================] - 0s 76ms/step - loss: 1.0269 - categorical_accuracy: 0.4737
Epoch 8/100
4/4 [==============================] - 0s 75ms/step - loss: 1.1634 - categorical_accuracy: 0.4825
Epoch 9/100
4/4 [==============================] - 0s 87ms/step - loss: 1.2770 - categorical_accuracy: 0.4298
Epoch 10/1

In [125]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_25 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_26 (LSTM)              (None, 64)                49408     
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dropout_11 (Dropout)        (None, 32)               

In [126]:
res = model.predict(X_test)

1/1 [==============================] - 0s 261ms/step


In [127]:
actions[np.argmax(res[0])]

'my'

In [128]:
actions[np.argmax(y_test[0])]

'my'

In [129]:
model.save('..//models//alphabets.h5')

In [130]:
yhat = model.predict(X_train)

4/4 [==============================] - 0s 27ms/step


In [131]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [132]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[84,  1],
        [ 0, 29]],

       [[85,  0],
        [ 0, 29]],

       [[85,  0],
        [ 1, 28]],

       [[87,  0],
        [ 0, 27]]], dtype=int64)

In [133]:
accuracy_score(ytrue, yhat)

0.9912280701754386